<a href="https://colab.research.google.com/github/Charmzyy/NlP/blob/main/Nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio -q
!pip install sentence-transformers -q
!pip install nltk -q

import nltk
nltk.download('stopwords')
nltk.download('punkt')

     |████████████████████████████████| 11.6 MB 5.2 MB/s 
     |████████████████████████████████| 84 kB 3.6 MB/s 
     |████████████████████████████████| 106 kB 52.6 MB/s 
     |████████████████████████████████| 84 kB 2.9 MB/s 
     |████████████████████████████████| 54 kB 1.9 MB/s 
     |████████████████████████████████| 272 kB 58.7 MB/s 
     |████████████████████████████████| 2.3 MB 51.9 MB/s 
     |████████████████████████████████| 213 kB 47.3 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 80 kB 8.2 MB/s 
     |████████████████████████████████| 68 kB 6.6 MB/s 
     |████████████████████████████████| 46 kB 4.1 MB/s 
     |████████████████████████████████| 4.1 MB 34.3 MB/s 
     |████████████████████████████████| 593 kB 31.6 MB/s 
     |████████████████████████████████| 856 kB 71.3 MB/s 
     |██████████████████████████

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import pandas as pd

from nltk.corpus import stopwords


from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

import warnings
warnings.filterwarnings('ignore')

stop = stopwords.words('english')
print(len(stop))

179


In [4]:
df = pd.read_csv('titles.csv')
df.sample(10)

,title
9002,Crosslingual Induction of Semantic Roles.
1882,An Optimal Task Assignment Policy and Performa...
2162,Identifiability in Causal Bayesian Networks: A...
3782,Efficient Collaborative Crowdsourcing.
6238,A Pipeline Framework for Dependency Parsing.
15397,Supporting Sandtray Therapy on an Interactive ...
8111,Solving Relational Similarity Problems Using t...
14816,Effects of head-mounted and scene-oriented vid...
5034,Solution Quality Improvements for Massively Mu...
15078,Noticing notice: a large-scale experiment on t...


In [5]:
sample = df.sample(50)


sample.head()

,title
4911,Appearance-Based Obstacle Detection with Monoc...
10303,Multi-commodity Trade Application to the Routi...
7666,The Role of Information Retrieval in Answering...
15257,"ApplianceReader: A Wearable, Crowdsourced, Vis..."
10022,Dynamic Ontology-Based Sensor Binding.


In [6]:
#reset index
sample = sample.reset_index(drop=True)
sample.head()

,title
0,Appearance-Based Obstacle Detection with Monoc...
1,Multi-commodity Trade Application to the Routi...
2,The Role of Information Retrieval in Answering...
3,"ApplianceReader: A Wearable, Crowdsourced, Vis..."
4,Dynamic Ontology-Based Sensor Binding.


In [10]:
# cleaning
sample['ctitle'] = sample['title'].str.replace('[^\w\s]','')
sample['ctitle'] = sample['ctitle'].str.replace('\d+', '')
sample['ctitle'] = sample['ctitle'].str.lower()

sample['ctitle'] = sample['ctitle'].apply(lambda item: " ".join(item for item in item.split() if item not in stop))


sample.head()

,title,ctitle
0,Appearance-Based Obstacle Detection with Monoc...,appearancebased obstacle detection monocular c...
1,Multi-commodity Trade Application to the Routi...,multicommodity trade application routing algor...
2,The Role of Information Retrieval in Answering...,role information retrieval answering complex q...
3,"ApplianceReader: A Wearable, Crowdsourced, Vis...",appliancereader wearable crowdsourced visionba...
4,Dynamic Ontology-Based Sensor Binding.,dynamic ontologybased sensor binding


In [11]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
#feature extraction

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
features = model.encode(sample['title'])
sample['features']=list(features)
sample.head()

In [ ]:
print(sample['title'].iloc[9])
print(sample['features'].iloc[2].shape)
#print(sample['features'].iloc[0])

In [18]:
sample_query =sample.iloc[0]['title']
sample_query


'Appearance-Based Obstacle Detection with Monocular Color Vision.'

In [19]:
query_feature = model.encode([sample_query])

In [20]:
sample['sim'] = sample['features'].apply(lambda x: cosine_similarity([x], query_feature)[0][0])
sample.head()

,title,ctitle,features,sim
0,Appearance-Based Obstacle Detection with Monoc...,appearancebased obstacle detection monocular c...,"[-0.9871944, -0.28354323, 0.76582485, -0.27368...",1.000000
1,Multi-commodity Trade Application to the Routi...,multicommodity trade application routing algor...,"[-1.0929424, -0.682992, 0.05620686, -0.2633092...",0.420233
2,The Role of Information Retrieval in Answering...,role information retrieval answering complex q...,"[-1.2407819, -0.11698342, 0.48437366, 0.507496...",0.577534
3,"ApplianceReader: A Wearable, Crowdsourced, Vis...",appliancereader wearable crowdsourced visionba...,"[-0.97534364, -0.5598161, 1.4497715, -0.237582...",0.587597
4,Dynamic Ontology-Based Sensor Binding.,dynamic ontologybased sensor binding,"[-0.941023, -0.26813403, 1.059211, -0.41262668...",0.574116


In [ ]:
sorted_sample = sample.sort_values(by='sim', ascending=False)
sorted_sample.tail()

In [22]:
def return_sim(query):
        query_feature  = model.encode([query])
        sample['sim'] = sample['features'].apply(lambda x: cosine_similarity([x], query_feature)[0][0])
        sorted_sample = sample.sort_values(by='sim', ascending=False)
        result = sorted_sample['title'].head(10)
        
        return list(result)


In [23]:
x = str(input("enter a query"))
print(return_sim(x))

enter a querytodat
['tRuEcasIng.', 'Topic-focus and Salience.', 'Backdoors to Planning.', 'A Generic Sentence Trimmer with CRFs.', 'Single-Frontier Bidirectional Search.', 'Non-Cooperation in Dialogue.', 'Trace Ratio Criterion for Feature Selection.', 'Compact CFR.', 'Single Document Summarization with Document Expansion.', 'General binarization for parsing and translation.']


In [26]:
import gradio as gr

iface = gr.Interface(fn=return_sim, inputs = "text", outputs = "text")
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://3375bc94018471dc.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7fbf1dcdcad0>,
 'http://127.0.0.1:7861/',
 'https://3375bc94018471dc.gradio.app')